# Лабораторная работа: Мета-обучение
## Шаг 2: Извлечение мета-признаков и эксперимент с перестановками

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

from src.meta_features.fixed_extractor import FixedMetaFeatureExtractor

%matplotlib inline

In [ ]:
# Загрузка Iris
iris = load_iris()
X, y = iris.data, iris.target
extractor = FixedMetaFeatureExtractor()

# Оригинал
original = extractor.extract(X, y, 'original')
print("Оригинальные мета-признаки:")
for k, v in list(original.items())[:8]:
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")
    else:
        print(f"  {k}: {v}")

In [ ]:
# Перемешивание строк
np.random.seed(42)
X_rows = X.copy()
np.random.shuffle(X_rows)
rows = extractor.extract(X_rows, y, 'rows')

# Перемешивание столбцов
X_cols = X.copy()
np.random.shuffle(X_cols.T)
cols = extractor.extract(X_cols, y, 'cols')

# Перестановка меток
y_perm = y.copy()
unique_classes = np.unique(y)
perm = np.random.permutation(unique_classes)
for old, new in zip(unique_classes, perm):
    y_perm[y == old] = new
labels = extractor.extract(X, y_perm, 'labels')

print("\nИзменения после перестановок:")
for feat in ['n_instances', 'mean_mean', 'class_entropy', 'correlation_mean']:
    if feat in original and feat in rows:
        print(f"\n{feat}:")
        print(f"  Оригинал: {original[feat]:.4f}")
        print(f"  Строки: {rows[feat]:.4f} (изм: {rows[feat]-original[feat]:+.4f})")
        print(f"  Столбцы: {cols[feat]:.4f} (изм: {cols[feat]-original[feat]:+.4f})")
        print(f"  Метки: {labels[feat]:.4f} (изм: {labels[feat]-original[feat]:+.4f})")

### Выводы:
- Перемешивание строк не меняет мета-признаки
- Перемешивание столбцов сильно меняет корреляции
- Перестановка меток влияет на энтропию классов